In [14]:
!pip install langchain
!pip install chromadb
!pip install llama-cpp-python
!pip install pandas
!pip install google-colab

In [15]:
!pip install -U langchain-community

In [16]:
!pip install sentence-transformers


In [ ]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import requests

prompt_template_questions = """
You are an expert in creating practice questions based on study material.
Your goal is to prepare a student for their exam. You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the student for their exam. Make sure not to lose any important information.

QUESTIONS:
"""
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template_questions, input_variables=["text"])

refine_template_questions = """
You are an expert in creating practice questions based on study material.
Your goal is to help a student prepare for an exam.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
"""

REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template_questions,
)


def download_model(url, dest_path):
    response = requests.get(url, stream=True)
    with open(dest_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

model_url = "https://huggingface.co/TheBloke/zephyr-7B-alpha-GGUF/resolve/main/zephyr-7b-alpha.Q4_K_M.gguf"
model_path = "zephyr-7b-alpha.Q4_K_M.gguf"

if not os.path.exists(model_path):
    print("Downloading model...")
    download_model(model_url, model_path)
    print("Model downloaded.")


text_question_gen = input("Please enter the passage you want to generate questions from: ")

text_splitter_question_gen = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
text_chunks_question_gen = text_splitter_question_gen.split_text(text_question_gen)
docs_question_gen = [Document(page_content=t) for t in text_chunks_question_gen]


try:

    llm_question_gen = LlamaCpp(
        streaming=True,
        model_path=model_path,
        temperature=0.75,
        top_p=1,
        verbose=False,
        n_ctx=4096
    )


    question_gen_chain = load_summarize_chain(
        llm=llm_question_gen,
        chain_type="refine",
        verbose=False,
        question_prompt=PROMPT_QUESTIONS,
        refine_prompt=REFINE_PROMPT_QUESTIONS
    )


    questions = question_gen_chain.run(docs_question_gen)
    print(questions)

    llm_answer_gen = LlamaCpp(
        streaming=True,
        model_path=model_path,
        temperature=0.75,
        top_p=1,
        verbose=False,
        n_ctx=4096
    )


    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
    vector_store = Chroma.from_documents(docs_question_gen, embeddings)


    answer_gen_chain = RetrievalQA.from_chain_type(
        llm=llm_answer_gen,
        chain_type="stuff",
        retriever=vector_store.as_retriever(k=2)
    )

    question_list = questions.split("\n")

    for question in question_list:
            print("Question: ", question)
            answer = answer_gen_chain.run(question)
            print("Answer: ", answer)
            print("--------------------------------------------------\n\n")

except Exception as e:
    print(f"An error occurred: {e}")


Please enter the passage you want to generate questions from: react is a javascript library 
Generated Questions:
 1. What is react? (True/False)
        A. React is a programming language used for web development.
        B. React is a javascript library used for web development.
    2. Can you describe the purpose of react?
    3. Which type of application does react specialize in creating?
    4. What are some benefits of using react over other libraries or frameworks?
    5. What are some common use cases for react in web applications?
    6. How does react handle component state?
    7. What is the main syntax used in react code?
    8. Can you explain what a JSX tag is and how it differs from regular HTML tags?
    9. How can you pass data to a child component in react?
    10. How do you handle events in react components?
    11. How does react handle server-side rendering?
    12. What is the difference between client-side and server-side rendering in react?
    13. How can you

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

Answer:    React uses JSX syntax, which combines JavaScript and HTML.
--------------------------------------------------

Question:     8. Can you explain what a JSX tag is and how it differs from regular HTML tags?


Answer:  Yes, I can help with that! In React, JSX (JavaScript XML) is used to write the views or the UI (User Interface). It allows you to combine JavaScript code inside your HTML-like syntax. The resulting component can then be reused and passed as a prop between components. For example, instead of writing:

```
var element = document.createElement('h1');
element.className = 'hello';
element.innerHTML = 'Hello, World! ';
document.body.appendChild(element);
```

In React, we can use JSX to do the same thing in a more concise way:

```
var element = <h1 className="hello">Hello, World!</h1>;
ReactDOM.render(element, document.body);
```

The syntax for JSX tags is very similar to regular HTML tags but with some differences. Here are a few examples:

- JSX tags are self-closing (for example, `<img src="myimage.jpg" />`)
- CamelCase instead of hyphens (for example, `className` instead of `class`)
- No need for the
--------------------------------------------------

Question:     9. How can 

Answer:  10. You can use props (short for properties) to pass data from parent to child components in React.
--------------------------------------------------

Question:     10. How do you handle events in react components?


Answer:  In React components, we use event handlers to handle user input or other events. We can define the event handler function as a regular function or as an arrow function and then bind it to the component using either the constructor method or the `bind` method provided by React. Then in our JSX code, we add an `onClick` attribute with the name of our event handler function followed by parentheses to pass any necessary arguments. For example, to handle a click event on a button that calls a function `myFunction()`, you would write:

```javascript
class MyComponent extends React.Component {
  constructor(props) {
    super(props);
    this.myFunction = this.myFunction.bind(this);
  }
  myFunction() {
    console.log('You clicked the button!');
  }
  render() {
    return (
      <div>
        <button onClick={this.myFunction}>Click me</button>
      </div>
    );
  }
}
```
Alternatively, you can use arrow functions to simplify the code:

```javascript
class MyComponent extends Rea

Answer:  React doesn't have built-in support for server-side rendering. However, it can be combined with other technologies like Node.js and Express to create a server-side rendering setup. This allows you to pre-render your app on the server before sending it to the client. The most popular library for this is Next.js.
--------------------------------------------------

Question:     12. What is the difference between client-side and server-side rendering in react?
